In [ ]:
# %tensorflow_version 2.x
# !pip uninstall -y tensorflow
# !pip install tensorflow-gpu==1.15.0

In [ ]:
import tensorflow as tf
import numpy as np
import scipy.io
import timeit
import sys
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import random
import seaborn as sns
import pandas as pd
from glob import glob 
import xlwt
import os 

In [ ]:
class neural_net(object):
    def __init__(self, *inputs, layers, monotonic = False, normalize = False, LAA = False):

        self.layers = layers
        self.num_layers = len(self.layers)
        self.monotonic = monotonic
        self.normalize = normalize
        self.LAA = LAA
        
        if self.normalize:
            if len(inputs) == 0:
                in_dim = self.layers[0]
                self.X_mean = np.zeros([1, in_dim])
                self.X_std = np.ones([1, in_dim])
            else:
                X = np.concatenate(inputs, 1)
                self.X_mean = X.mean(0, keepdims=True)
                self.X_std = X.std(0, keepdims=True)

        self.weights = []
        self.biases = []
        self.A = []

        for l in range(0,self.num_layers-1):
            in_dim = self.layers[l]
            out_dim = self.layers[l+1]
            xavier_stddev = np.sqrt(2/(in_dim + out_dim))
            W = tf.Variable(tf.random.truncated_normal([in_dim, out_dim], stddev = xavier_stddev),dtype=tf.float32, trainable=True) 
            # monotonically increasing neural network
            if self.monotonic:
                W = W**2
            b = tf.Variable(np.zeros([1, out_dim]), dtype=tf.float32, trainable=True)
            
            # tensorflow variables
            self.weights.append(W)
            self.biases.append(b)
            
            # locally adaptive activation
            if self.LAA:
                a = tf.Variable(0.05, dtype=tf.float32)
                self.A.append(a)
            
    def __call__(self, *inputs):
        if self.normalize:
            H = (tf.concat(inputs, 1) - self.X_mean)/self.X_std
        else:
            H = tf.concat(inputs, 1)

        for l in range(0, self.num_layers-1):
            W = self.weights[l]
            b = self.biases[l]
            
            # matrix multiplication
            H = tf.matmul(H, W)
            # add bias
            H = H + b
            # activation
            if l < self.num_layers-2:
                if self.LAA:
                    H = tf.tanh(20 * self.A[l]*H)
                else:
                    H = tf.tanh(H)

        return H

In [ ]:
def tf_session():
    # tf session
    config = tf.ConfigProto(allow_soft_placement=True,  # an operation might be placed on CPU for example if GPU is not available
                            log_device_placement=True)  # print out device information
    config.gpu_options.allow_growth = True
    sess = tf.Session(config=config)

    # init
    init = tf.global_variables_initializer()
    sess.run(init)

    return sess

In [ ]:
def VGM(psi, theta_r, theta_s, alpha, n, K_s, l):
    """
    this function calculates volumetric water content and hydraulic conductivity
    from matric potential (psi: negative) given VGM (van-Genuchen and Mualem model) parameters
    output: (theta, K)
    theta: volumetric water content
    K: hydraulic conductivity
    By default, alpha, n, and K_s are the parameters to be estiated.
    """
    m = 1 - 1/n
    S_e = (1 + (-alpha*psi)**n)**(-m)
    theta =  S_e * (theta_s - theta_r) + theta_r
    K = K_s*S_e**l*(1-(1-S_e**(1/m))**m)**2
    return theta, K

In [ ]:
# # theta_r, theta_s, alpha [1/cm], n, K_s [cm/hour], l
# layer 1 is the lower layer while layer 2 is the upper layer
VGM_parameters = {
            "layer2": [0.078, 0.43, 0.036, 1.56, 1.04, 0.5],
            "layer1": [0.065, 0.41, 0.075, 1.89, 4.42083, 0.5]}

In [ ]:
# sess = tf_session()

# log10_h = np.arange(-2, 6, 0.1)  # logarithm of suction (negative matric potential) in cm
# psi = -10**log10_h

# tf_psi = tf.constant(psi, dtype = tf.float32)

# tf_theta_1, tf_K_1 = VGM(tf_psi, *VGM_parameters["layer1"])
# tf_theta_2, tf_K_2 = VGM(tf_psi, *VGM_parameters["layer2"])
# psi = sess.run(tf_psi)
# theta_1 = sess.run(tf_theta_1)
# theta_2 = sess.run(tf_theta_2)
# K_1 = sess.run(tf_K_1)
# K_2 = sess.run(tf_K_2)

In [ ]:
# plt.plot(np.log10(-psi), theta_1, label = "layer1")
# plt.plot(np.log10(-psi), theta_2, label =  "layer2")
# plt.title("Water Retention Curve")
# plt.legend()
# plt.xlabel("log$_{10}(-\\psi)$")
# plt.ylabel("$\\theta$")

In [ ]:
# plt.plot(np.log10(-psi), K_1, label = "layer 1")
# plt.plot(np.log10(-psi), K_2, label = "layer 2")
# plt.title("Hydraulic Conductivity Function")
# plt.legend()
# plt.xlabel("log$_{10}(-\\psi)$")
# plt.ylabel("$K [cm / hour]$")

In [ ]:
def log10(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator

In [ ]:
def RRE_mixed_1D(psi, theta, K, t, z):
    
    theta_t = tf.gradients(theta, t)[0]
    theta_z = tf.gradients(theta, z)[0]
    
    psi_z = tf.gradients(psi, z)[0]
    psi_zz = tf.gradients(psi_z, z)[0]
    
    K_z = tf.gradients(K, z)[0]

    qz = -K*(psi_z + 1.0)
    residual = theta_t - K_z*psi_z - K*psi_zz - K_z
    return residual, qz

In [ ]:
class Sampler:
    # Initialize the class
    def __init__(self, dim, coords, func, name = None):
        self.dim = dim
        self.coords = coords
        self.func = func
        self.name = name
    def sample(self, N):
        x = self.coords[0:1,:] + (self.coords[1:2,:]-self.coords[0:1,:])*np.random.rand(N, self.dim)
        y = self.func(x)
        return x, y

In [ ]:
class PINNs_RRE1D(object):
    """
    PINNs_RRE1D class implements physics-informed neural network solver for the inverse modeling of the
    Ricahrdson-Ricahrds equation using TensorFlow 1.15.
    # notational conventions
    _tf: placeholders
    _pred: output of neural networks or other parametric models
    _res: values related to residual points
    _data: training data 
    _ic: initial condition
    _bc: boundary condition (ub: upper; lb: lower for 1D)
    _star: prediction (used after training)
    _1: lower layer
    _2: upper layer
    _int: interface of the layers
    t: time
    z: vertical coordinate
    theta: volumetric water content
    psi: matric potential
    K: hydraulic conductivity
    ALR: adaptive learning rate
    LAA: layer-wise locally adaptive activation functions
    weight_values: [data_1, data_2, res_2, res_int, psi_int, qz_int]
    """
    def __init__(self, t_res_1, z_res_1,
                       t_res_2, z_res_2,
                       t_int, z_int,
                       t_data_1, z_data_1, theta_data_1,
                       t_data_2, z_data_2, theta_data_2,
                       layers_psi_1, layers_psi_2, hydraulic_model, 
                       hydraulic_parameters_1, hydraulic_parameters_2, weight_values,
                       normalize, ALR, LAA):
        
        # NN architecture
        self.layers_psi_1 = layers_psi_1
        self.layers_psi_2 = layers_psi_2
        
        # Mode 
        self.hydraulic_model = hydraulic_model  # "monotonic", "VGM", "Brooks","PDI", "Gardner"
        self.normalize = normalize # normalize input values to neural networks
        self.ALR = ALR  # adaptive learning rate
        self.LAA = LAA  # locally adaptive activation functions
        
        # Adaptive re-weighting constant
        self.rate = 0.9
        
        # parameter for the output of neural networks
        self.beta = 0.0
        
        # initial values for weights in the loss function
        
        self.adaptive_constant_data_1_val = np.array(weight_values[0])
        self.adaptive_constant_data_2_val = np.array(weight_values[1])
        self.adaptive_constant_res_2_val = np.array(weight_values[2])
        self.adaptive_constant_res_int_val = np.array(weight_values[3])
        self.adaptive_constant_psi_int_val = np.array(weight_values[4])
        self.adaptive_constant_qz_int_val = np.array(weight_values[5])
        
        # hydraulic parameters  
                    
        if self.hydraulic_model == "VGM":  # theta_r, theta_s, alpha, n, K_s, l
            self.theta_r_1 = tf.Variable(hydraulic_parameters_1[0], dtype=tf.float32, trainable=False)
            self.theta_s_1 = tf.Variable(hydraulic_parameters_1[1], dtype=tf.float32, trainable=False)
            self.alpha_1 = tf.Variable(hydraulic_parameters_1[2], dtype=tf.float32, trainable=False)
            self.n_1 = tf.Variable(hydraulic_parameters_1[3], dtype=tf.float32, trainable=False)
            self.K_s_1 = tf.Variable(hydraulic_parameters_1[4], dtype=tf.float32, trainable=False)
            self.l_1 = tf.Variable(hydraulic_parameters_1[5], dtype=tf.float32, trainable=False)
            
            self.theta_r_2 = tf.Variable(hydraulic_parameters_2[0], dtype=tf.float32, trainable=False)
            self.theta_s_2 = tf.Variable(hydraulic_parameters_2[1], dtype=tf.float32, trainable=False)
            self.alpha_2 = tf.Variable(hydraulic_parameters_2[2], dtype=tf.float32, trainable=False)
            self.n_2 = tf.Variable(hydraulic_parameters_2[3], dtype=tf.float32, trainable=False)
            self.K_s_2 = tf.Variable(hydraulic_parameters_2[4], dtype=tf.float32, trainable=False)
            self.l_2 = tf.Variable(hydraulic_parameters_2[5], dtype=tf.float32, trainable=False)

        # data
        
        [self.t_res_1, self.z_res_1] = [t_res_1, z_res_1]
        [self.t_res_2, self.z_res_2] = [t_res_2, z_res_2]
        [self.t_int, self.z_int] = [t_int, z_int]
        [self.t_data_1, self.z_data_1, self.theta_data_1] = [t_data_1, z_data_1, theta_data_1]
        [self.t_data_2, self.z_data_2, self.theta_data_2] = [t_data_2, z_data_2, theta_data_2]
        
        # placeholders
        
        [self.t_res_1_tf, self.z_res_1_tf] = [tf.placeholder(tf.float32, shape=[None, 1]) for _ in range(2)]
        [self.t_res_2_tf, self.z_res_2_tf] = [tf.placeholder(tf.float32, shape=[None, 1]) for _ in range(2)]
        [self.t_int_tf, self.z_int_tf] = [tf.placeholder(tf.float32, shape=[None, 1]) for _ in range(2)]

        [self.t_data_1_tf, self.z_data_1_tf, self.theta_data_1_tf] = [tf.placeholder(tf.float32, shape=[None, 1]) for _ in range(3)]
        [self.t_data_2_tf, self.z_data_2_tf, self.theta_data_2_tf] = [tf.placeholder(tf.float32, shape=[None, 1]) for _ in range(3)]
                
        # weight parameters in the loss function
        self.adaptive_constant_data_1_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_data_1_val.shape)
        self.adaptive_constant_data_2_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_data_2_val.shape)
        self.adaptive_constant_res_2_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_res_2_val.shape)
        self.adaptive_constant_res_int_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_res_int_val.shape)
        self.adaptive_constant_psi_int_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_psi_int_val.shape)
        self.adaptive_constant_qz_int_tf = tf.placeholder(tf.float32, shape=self.adaptive_constant_qz_int_val.shape)
        
        # neural network definition
        self.net_psi_1 = neural_net(self.t_res_1, self.z_res_1, layers = self.layers_psi_1, monotonic = False, normalize = self.normalize, LAA = self.LAA)
        self.net_psi_2 = neural_net(self.t_res_2, self.z_res_2, layers = self.layers_psi_2, monotonic = False, normalize = self.normalize, LAA = self.LAA)
     
        # prediction at data points
        
        self.log_psi_data_1_pred = self.net_psi_1(self.t_data_1_tf, self.z_data_1_tf)
        self.psi_data_1_pred = -tf.exp(self.log_psi_data_1_pred) + self.beta
        self.theta_data_1_pred, self.K_data_1_pred = VGM(self.psi_data_1_pred, self.theta_r_1, self.theta_s_1, self.alpha_1, self.n_1, self.K_s_1, self.l_1)

        self.log_psi_data_2_pred = self.net_psi_2(self.t_data_2_tf, self.z_data_2_tf)
        self.psi_data_2_pred = -tf.exp(self.log_psi_data_2_pred) + self.beta
        self.theta_data_2_pred, self.K_data_2_pred = VGM(self.psi_data_2_pred, self.theta_r_2, self.theta_s_2, self.alpha_2, self.n_2, self.K_s_2, self.l_2)
                
        # prediction at residual points
        self.log_psi_res_1_pred = self.net_psi_1(self.t_res_1_tf, self.z_res_1_tf)
        self.psi_res_1_pred = -tf.exp(self.log_psi_res_1_pred) + self.beta
        self.theta_res_1_pred, self.K_res_1_pred = VGM(self.psi_res_1_pred, self.theta_r_1, self.theta_s_1, self.alpha_1, self.n_1, self.K_s_1, self.l_1)
              
        self.residual_res_1_pred, self.qz_res_1_pred = RRE_mixed_1D(self.psi_res_1_pred, self.theta_res_1_pred, self.K_res_1_pred,
                                                                self.t_res_1_tf, self.z_res_1_tf)
        
        self.log_psi_res_2_pred = self.net_psi_2(self.t_res_2_tf, self.z_res_2_tf)
        self.psi_res_2_pred = -tf.exp(self.log_psi_res_2_pred) + self.beta
        self.theta_res_2_pred, self.K_res_2_pred = VGM(self.psi_res_2_pred, self.theta_r_2, self.theta_s_2, self.alpha_2, self.n_2, self.K_s_2, self.l_2)
              
        self.residual_res_2_pred, self.qz_res_2_pred = RRE_mixed_1D(self.psi_res_2_pred, self.theta_res_2_pred, self.K_res_2_pred,
                                                                self.t_res_2_tf, self.z_res_2_tf)

        # psi, flux, and residual at the layer interface
        # layer 1
        self.log_psi_int_1_pred = self.net_psi_1(self.t_int_tf, self.z_int_tf)
        self.psi_int_1_pred = -tf.exp(self.log_psi_int_1_pred) + self.beta
        self.theta_int_1_pred, self.K_int_1_pred = VGM(self.psi_int_1_pred, self.theta_r_1, self.theta_s_1, self.alpha_1, self.n_1, self.K_s_1, self.l_1)
              
        self.residual_int_1_pred, self.qz_int_1_pred = RRE_mixed_1D(self.psi_int_1_pred, self.theta_int_1_pred, self.K_int_1_pred,
                                                                self.t_int_tf, self.z_int_tf)
        
        # layer 2
        self.log_psi_int_2_pred = self.net_psi_2(self.t_int_tf,  self.z_int_tf)
        self.psi_int_2_pred = -tf.exp(self.log_psi_int_2_pred) + self.beta
        self.theta_int_2_pred, self.K_int_2_pred = VGM(self.psi_int_2_pred, self.theta_r_2, self.theta_s_2, self.alpha_2, self.n_2, self.K_s_2, self.l_2)
              
        self.residual_int_2_pred, self.qz_int_2_pred = RRE_mixed_1D(self.psi_int_2_pred, self.theta_int_2_pred, self.K_int_2_pred,
                                                                self.t_int_tf, self.z_int_tf)
                
        # loss
        # layer 1
        self.loss_res_1 =  tf.reduce_mean(tf.square(self.residual_res_1_pred))
        self.loss_data_1 = tf.reduce_mean(tf.square(self.theta_data_1_pred - self.theta_data_1_tf))
        
        self.loss_1 = self.loss_res_1 +  self.adaptive_constant_data_1_tf * self.loss_data_1 
        
        # layer 2
        self.loss_res_2 =  tf.reduce_mean(tf.square(self.residual_res_2_pred))
        self.loss_data_2 = tf.reduce_mean(tf.square(self.theta_data_2_pred - self.theta_data_2_tf))
                
        self.loss_2 = self.adaptive_constant_res_2_tf * self.loss_res_2 +  self.adaptive_constant_data_2_tf * self.loss_data_2
        
        self.loss_res_int = tf.reduce_mean(tf.square(self.residual_int_1_pred - self.residual_int_2_pred))
        self.loss_psi_int = tf.reduce_mean(tf.square(self.log_psi_int_1_pred - self.log_psi_int_2_pred))
        self.loss_qz_int = tf.reduce_mean(tf.square(self.qz_int_1_pred - self.qz_int_2_pred))
        
        self.loss_int = self.adaptive_constant_res_int_tf * self.loss_res_int  \
                         +  self.adaptive_constant_psi_int_tf * self.loss_psi_int  \
                         +  self.adaptive_constant_qz_int_tf * self.loss_qz_int
        
        
        self.loss = self.loss_1 + self.loss_2 + self.loss_int
        
                    
        # Define optimizer with learning rate schedule
        self.global_step = tf.Variable(0, trainable = False)
        starter_learning_rate = 1e-3
        self.learning_rate = tf.train.exponential_decay(starter_learning_rate, self.global_step,
                                                        1000, 0.90, staircase=False)
        
        # Passing global_step to minimize() will increment it at each step.
        # self.train_op_1 = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss_1, global_step=self.global_step)
        # self.train_op_2 = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss_2, global_step=self.global_step)
        self.train_op = tf.train.AdamOptimizer(self.learning_rate).minimize(self.loss, global_step=self.global_step)
        

        # L-BFGS-B method
        self.optimizer = tf.contrib.opt.ScipyOptimizerInterface(self.loss,
                                                        method = 'L-BFGS-B',
                                                        options = {'maxiter': 50000,
                                                                   'maxfun': 50000,
                                                                   'maxcor': 50,
                                                                   'maxls': 50,
                                                                   'ftol' : 1e-10,
                                                                   'gtol' : 1e-8})
        
        
        # Logger
        
        self.loss_res_1_log = []
        self.loss_data_1_log = []
        self.loss_res_2_log = []
        self.loss_data_2_log = []
        self.loss_psi_int_log = []
        self.loss_qz_int_log = []
        self.loss_res_int_log = []
        
        self.loss_res_1_log_BFGS = []
        self.loss_data_1_log_BFGS = []
        self.loss_res_2_log_BFGS = []
        self.loss_data_2_log_BFGS = []
        self.loss_psi_int_log_BFGS = []
        self.loss_qz_int_log_BFGS = []
        self.loss_res_int_log_BFGS = []
        
        self.saver = tf.train.Saver()
        
        # new residual points
        self.new_residual_1 = []
        self.new_residual_2 = []
        
        # Store the adaptive constant
        
        self.adaptive_constant_data_1_log = []
        
        self.adaptive_constant_res_2_log = []
        self.adaptive_constant_data_2_log = []
        
        self.adaptive_constant_res_int_log = []
        self.adaptive_constant_psi_int_log = []
        self.adaptive_constant_qz_int_log = []
        
                
        # Compute the adaptive constant
        
        self.adaptive_constant_data_1_list = []
        
        self.adaptive_constant_res_2_list = []
        self.adaptive_constant_data_2_list = []
        
        self.adaptive_constant_res_int_list = []
        self.adaptive_constant_psi_int_list = []
        self.adaptive_constant_qz_int_list = []
        
        
        self.sess = tf_session()

    def train_Adam(self, iterations, batch = True, batch_size = 128):

        N_res_1 = self.t_res_1.shape[0]
        N_res_2 = self.t_res_2.shape[0]

        start_time = timeit.default_timer()
        running_time = 0
        it = 0
        for it in range(iterations):
           
            if batch:

                idx_res_1 = np.random.choice(N_res_1, batch_size, replace = False)
                idx_res_2 = np.random.choice(N_res_2, batch_size, replace = False)

                (t_res_1, z_res_1) = (self.t_res_1[idx_res_1,:],
                                  self.z_res_1[idx_res_1,:])
                (t_res_2, z_res_2) = (self.t_res_2[idx_res_2,:],
                                  self.z_res_2[idx_res_2,:])
                
            else:

                (t_res_1, z_res_1) = (self.t_res_1,
                                  self.z_res_1)
                (t_res_2, z_res_2) = (self.t_res_2,
                                  self.z_res_2)

            tf_dict = {self.t_res_1_tf: t_res_1,
                       self.z_res_1_tf: z_res_1,
                       self.t_res_2_tf: t_res_2,
                       self.z_res_2_tf: z_res_2,
                       self.t_int_tf: self.t_int,
                       self.z_int_tf: self.z_int,
                       self.t_data_1_tf: self.t_data_1,
                       self.z_data_1_tf: self.z_data_1,
                       self.theta_data_1_tf: self.theta_data_1,
                       self.t_data_2_tf: self.t_data_2,
                       self.z_data_2_tf: self.z_data_2,
                       self.theta_data_2_tf: self.theta_data_2,
                       self.adaptive_constant_data_1_tf: self.adaptive_constant_data_1_val,
                       self.adaptive_constant_res_2_tf: self.adaptive_constant_res_2_val,
                       self.adaptive_constant_data_2_tf: self.adaptive_constant_data_2_val,
                       self.adaptive_constant_res_int_tf: self.adaptive_constant_res_int_val,
                       self.adaptive_constant_psi_int_tf: self.adaptive_constant_psi_int_val,
                       self.adaptive_constant_qz_int_tf: self.adaptive_constant_qz_int_val}
        
        
#             self.sess.run(self.train_op_1, tf_dict)
#             self.sess.run(self.train_op_2, tf_dict)
            self.sess.run(self.train_op, tf_dict)
    
            # Print
            if it % 10 == 0:
                elapsed = timeit.default_timer() - start_time
                running_time += elapsed/3600.0
                
                
        
        
                loss_res_1_value, loss_data_1_value = self.sess.run([self.loss_res_1, self.loss_data_1], tf_dict)
                loss_res_2_value, loss_data_2_value = self.sess.run([self.loss_res_2, self.loss_data_2], tf_dict)

                loss_psi_int_value, loss_qz_int_value, loss_res_int_value = self.sess.run([self.loss_psi_int, self.loss_qz_int, self.loss_res_int], tf_dict)
                                                                                               
                self.loss_res_1_log.append(loss_res_1_value)
                self.loss_data_1_log.append(loss_data_1_value)
                self.loss_res_2_log.append(loss_res_2_value)
                self.loss_data_2_log.append(loss_data_2_value)
                self.loss_psi_int_log.append(loss_psi_int_value)
                self.loss_qz_int_log.append(loss_qz_int_value)
                self.loss_res_int_log.append(loss_res_int_value)   
                
                print('It: %d, Loss_res_1: %.3e, Loss_data_1: %.3e,  \
                               Loss_res_2: %.3e, Loss_data_2: %.3e,  \
                               Loss_psi_int: %.3e, Loss_qz_int: %.3e, Loss_res_int: %.3e, Time: %.2f' %
                      (it, loss_res_1_value, loss_data_1_value,
                           loss_res_2_value, loss_data_2_value,
                           loss_psi_int_value, loss_qz_int_value, loss_res_int_value, elapsed))
                
                start_time = timeit.default_timer()
                                               
    
    def train_BFGS(self):
        # full batch L-BFGS
        tf_dict = {self.t_res_1_tf: self.t_res_1,
                       self.z_res_1_tf: self.z_res_1,
                       self.t_res_2_tf: self.t_res_2,
                       self.z_res_2_tf: self.z_res_2,
                       self.t_int_tf: self.t_int,
                       self.z_int_tf: self.z_int,
                       self.t_data_1_tf: self.t_data_1,
                       self.z_data_1_tf: self.z_data_1,
                       self.theta_data_1_tf: self.theta_data_1,
                       self.t_data_2_tf: self.t_data_2,
                       self.z_data_2_tf: self.z_data_2,
                       self.theta_data_2_tf: self.theta_data_2,
                       self.adaptive_constant_data_1_tf: self.adaptive_constant_data_1_val,
                       self.adaptive_constant_res_2_tf: self.adaptive_constant_res_2_val,
                       self.adaptive_constant_data_2_tf: self.adaptive_constant_data_2_val,
                       self.adaptive_constant_res_int_tf: self.adaptive_constant_res_int_val,
                       self.adaptive_constant_psi_int_tf: self.adaptive_constant_psi_int_val,
                       self.adaptive_constant_qz_int_tf: self.adaptive_constant_qz_int_val}
        
#         L-BFGS-B
        self.optimizer.minimize(self.sess,
                                feed_dict = tf_dict,
                                fetches = [self.loss_res_1, self.loss_data_1,
                                           self.loss_res_2, self.loss_data_2,
                                           self.loss_psi_int, self.loss_qz_int, self.loss_res_int],
                                loss_callback = self.callback)
        
    def callback(self, loss_res_1_value, loss_data_1_value,
                           loss_res_2_value, loss_data_2_value,
                           loss_psi_int_value, loss_qz_int_value, loss_res_int_value):
        
        tf_dict = {self.t_res_1_tf: self.t_res_1,
                       self.z_res_1_tf: self.z_res_1,
                       self.t_res_2_tf: self.t_res_2,
                       self.z_res_2_tf: self.z_res_2,
                       self.t_int_tf: self.t_int,
                       self.z_int_tf: self.z_int,
                       self.t_data_1_tf: self.t_data_1,
                       self.z_data_1_tf: self.z_data_1,
                       self.theta_data_1_tf: self.theta_data_1,
                       self.t_data_2_tf: self.t_data_2,
                       self.z_data_2_tf: self.z_data_2,
                       self.theta_data_2_tf: self.theta_data_2,
                       self.adaptive_constant_data_1_tf: self.adaptive_constant_data_1_val,
                       self.adaptive_constant_res_2_tf: self.adaptive_constant_res_2_val,
                       self.adaptive_constant_data_2_tf: self.adaptive_constant_data_2_val,
                       self.adaptive_constant_res_int_tf: self.adaptive_constant_res_int_val,
                       self.adaptive_constant_psi_int_tf: self.adaptive_constant_psi_int_val,
                       self.adaptive_constant_qz_int_tf: self.adaptive_constant_qz_int_val}
        
        self.loss_res_1_log_BFGS.append(loss_res_1_value)
        self.loss_data_1_log_BFGS.append(loss_data_1_value)
        self.loss_res_2_log_BFGS.append(loss_res_2_value)
        self.loss_data_2_log_BFGS.append(loss_data_2_value)
        self.loss_psi_int_log_BFGS.append(loss_psi_int_value)
        self.loss_qz_int_log_BFGS.append(loss_qz_int_value)
        self.loss_res_int_log_BFGS.append(loss_res_int_value)   
                
                
        print('Loss_res_1: %.3e, Loss_data_1: %.3e,  \n\
                               Loss_res_2: %.3e, Loss_data_2: %.3e,  \n\
                               Loss_psi_int: %.3e, Loss_qz_int: %.3e, Loss_res_int: %.3e' %
                      (loss_res_1_value, loss_data_1_value,
                           loss_res_2_value, loss_data_2_value,
                           loss_psi_int_value, loss_qz_int_value, loss_res_int_value))
        
        
    def RAR(self, dom_coords_1, dom_coords_2, iterations, batchsize, m):
        self.train_Adam(iterations, batchsize)
        
        i = 1
        
        while i < 10:
            res_sampler_1 = Sampler(2, dom_coords_1, lambda x: 0)
            res_sampler_2 = Sampler(2, dom_coords_2, lambda x: 0)
            Z_res_1, _ = res_sampler_1.sample(np.int32(1e6))
            Z_res_2, _ = res_sampler_2.sample(np.int32(1e6))
            z_res_1, t_res_1 = Z_res_1[:, 0:1], Z_res_1[:, 1:2]
            z_res_2, t_res_2 = Z_res_2[:, 0:1], Z_res_2[:, 1:2]
        
            tf_dict = {self.t_res_1_tf: t_res_1,
                       self.z_res_1_tf: z_res_1,
                       self.t_res_2_tf: t_res_2,
                       self.z_res_2_tf: z_res_2}
            residual_1 = self.sess.run(self.residual_res_1_pred, tf_dict)
            residual_2 = self.sess.run(self.residual_res_2_pred, tf_dict)

            err_abs_1 = np.abs(residual_1)
            err_abs_2 = np.abs(residual_2)
            err_1 = np.mean(err_abs_1)
            err_2 = np.mean(err_abs_2)
            print("Mean residual for layer 1: %.3e, Mean residual for layer 2: %.3e" % (err_1, err_2))

            # choose m largest residual
            z_id_1 = err_abs_1.argsort(axis = 0)[-m * i:][::-1][:,0]
            z_id_2 = err_abs_2.argsort(axis = 0)[-m * i:][::-1][:,0]

            print(z_id_1, z_id_2)
            print("Adding new point for layer 1:", Z_res_1[z_id_1, :], "\n")
            print("High residuals for layer 1 are:", err_abs_1[z_id_1], "\n")
            print("Adding new point for layer 2:", Z_res_2[z_id_2, :], "\n")
            print("High residuals for layer 2 are:", err_abs_2[z_id_2], "\n")
        
            self.new_residual_1.append(Z_res_1[z_id_1, :])
            self.new_residual_2.append(Z_res_2[z_id_2, :])
            
            self.z_res_1 = np.vstack((self.z_res_1, Z_res_1[z_id_1, 0:1]))
            self.t_res_1 = np.vstack((self.t_res_1, Z_res_1[z_id_1, 1:2]))
            
            self.z_res_2 = np.vstack((self.z_res_2, Z_res_2[z_id_2, 0:1]))
            self.t_res_2 = np.vstack((self.t_res_2, Z_res_2[z_id_2, 1:2]))
            
            self.train_Adam(iterations, batchsize)
            
            i += 1

            print(i)
            
    def predict_1(self, t_star, z_star):

        tf_dict = {self.t_res_1_tf: t_star, self.z_res_1_tf: z_star}

        psi_star = self.sess.run(self.psi_res_1_pred, tf_dict)
        theta_star = self.sess.run(self.theta_res_1_pred, tf_dict)
        K_star = self.sess.run(self.K_res_1_pred, tf_dict)
        qz_star = self.sess.run(self.qz_res_1_pred, tf_dict)
        
        residual = self.sess.run(self.residual_res_1_pred, tf_dict)

        return psi_star, theta_star, K_star, qz_star, residual
    
    def predict_2(self, t_star, z_star):

        tf_dict = {self.t_res_2_tf: t_star, self.z_res_2_tf: z_star}

        psi_star = self.sess.run(self.psi_res_2_pred, tf_dict)
        theta_star = self.sess.run(self.theta_res_2_pred, tf_dict)
        K_star = self.sess.run(self.K_res_2_pred, tf_dict)
        qz_star = self.sess.run(self.qz_res_2_pred, tf_dict)
        
        residual = self.sess.run(self.residual_res_2_pred, tf_dict)

        return psi_star, theta_star, K_star, qz_star, residual

In [ ]:
# dom_coords_1 = np.array([[0.0, 0.0],   # t = 0 hour to t = 20 hours
#                        [20.0, -10.0]])
# dom_coords_2 = np.array([[0.0, 10.0],   # t = 0 hour to t = 20 hours
#                        [20.0, 0.0]])
# int_coords = np.array([[0.0, 0.0],
#                        [20.0, 0.0]])

In [ ]:
# produce residual points 
# res_sampler_1 = Sampler(2, dom_coords_1, lambda x: 0)
# res_sampler_2 = Sampler(2, dom_coords_2, lambda x: 0)
# int_sampler = Sampler(2, int_coords, lambda x: 0)
# # np.random.seed(0)
# Z_res_1, _ = res_sampler_1.sample(np.int32(100000))
# Z_res_2, _ = res_sampler_2.sample(np.int32(100000))
# Z_int, _ = int_sampler.sample(np.int32(10000))
# np.save("data/inverse_residual_points_layer1", Z_res_1)
# np.save("data/inverse_residual_points_layer2", Z_res_2)
# np.save("data/inverse_residual_points_interface", Z_int)

In [ ]:
# create noisy data
# theta_true = np.load(f"../analytical_solutions/hydrus_example_theta.npy")
# np.random.seed(0)

# noise = 0.005
# noise_theta = noise*np.random.randn(theta_true.shape[0], theta_true.shape[1])
# theta_noise = theta_true + noise_theta

# np.save("../analytical_solutions/hydrus_example_noisy_theta.npy", theta_noise)

In [ ]:
def main_loop(folder, ID, random_seed, data_depth, number_int = 1000, LAA = True, ALR = False, weight_values = [10, 10, 10, 1, 1, 1],
              number_layers_1 = 5, number_units_1 = 50, number_layers_2 = 5, number_units_2 = 50, 
              number_res_1 = 10000, number_res_2 = 10000,
              RAR = False, iterations = 100000, batch_size = 128, new_residual_points = 10):
    """
    ALR: adaptive learning rate
    LAA: layer-wise locally adaptive activation functions
    RAR: residual adaptive refinement
    weight_values: [data_1, data_2, res_2, res_int, psi_int, qz_int]
    data_depth: mm from the top of the soil (e.g., 10 means 1 cm from the top)
    """
    
    if not os.path.isdir(f'../results/{folder}'):
        os.mkdir(f'../results/{folder}')

    if not os.path.isdir(f'../results/{folder}/{ID}'):
        os.mkdir(f'../results/{folder}/{ID}')

    # domain boundaries; the first column is time, the second column is depth (positive upward)

    dom_coords_1 = np.array([[0.0, 0.0],   
                           [20.0, -10.0]])
    dom_coords_2 = np.array([[0.0, 10.0],  
                           [20.0, 0.0]])
    int_coords = np.array([[0.0, 0.0],
                           [20.0, 0.0]])

    # import residual points 
    Z_res_1 = np.load(f"data/inverse_residual_points_layer1.npy")
    Z_res_2 = np.load(f"data/inverse_residual_points_layer2.npy")
    Z_int = np.load(f"data/inverse_residual_points_interface.npy")

    # hydraulic paramters
    hydraulic_parameters_1 = VGM_parameters["layer1"]
    hydraulic_parameters_2 = VGM_parameters["layer2"]


    # hydralic function model
    hydraulic_model = "VGM"

    # normalization
    normalize = False

    # NN architecture
    layers_psi_1 = [2] + number_layers_1*[number_units_1] + [1]
    layers_psi_2 = [2] + number_layers_2*[number_units_2] + [1]

    # forward solution
    # collocation points
    if RAR:
        number_res_1_2 =  number_res_1 - (RAR_iterations - 1) * 10
        number_res_2_2 =  number_res_2 - (RAR_iterations - 1) * 10
    else:
        number_res_1_2 = number_res_1
        number_res_2_2 = number_res_2
        
    t_res_1 = Z_res_1[:number_res_1_2, 0:1]
    z_res_1 = Z_res_1[:number_res_1_2, 1:2]

    t_res_2 = Z_res_2[:number_res_2_2, 0:1]
    z_res_2 = Z_res_2[:number_res_2_2, 1:2]

    t_res = np.concatenate((t_res_2, t_res_1), axis = 1)
    z_res = np.concatenate((z_res_2, z_res_1), axis = 1) 
    
    # the first column is the top node 
    theta_true = np.load(f"../analytical_solutions/hydrus_example_theta.npy")
    theta_noise = np.load(f"../analytical_solutions/hydrus_example_noisy_theta.npy")
    data_depth_1 = [depth for depth in data_depth if depth > 100]
    data_depth_2 = [depth for depth in data_depth if depth < 100]

    theta_data_1 = theta_noise[:, [i + 1 for i in data_depth_1]].flatten(order = 'F')[:, None]
    theta_data_2 = theta_noise[:, data_depth_2].flatten(order = 'F')[:, None]
    
    T = 201
    N = 101
    t_1 = np.linspace(dom_coords_1[0, 0], dom_coords_1[1, 0], T)[:, None]
    z_1 = np.linspace(dom_coords_1[0, 1], dom_coords_1[1, 1], N)[:, None]
    z_1_mesh, t_1_mesh = np.meshgrid(z_1, t_1)

    t_2 = np.linspace(dom_coords_2[0, 0], dom_coords_2[1, 0], T)[:, None]
    z_2 = np.linspace(dom_coords_2[0, 1], dom_coords_2[1, 1], N)[:, None]
    z_2_mesh, t_2_mesh = np.meshgrid(z_2, t_2)

    t_data_1 = t_1_mesh[:,[i - 100 for i in data_depth_1]].flatten(order = 'F')[:, None]
    t_data_2 = t_2_mesh[:,data_depth_2].flatten(order = 'F')[:, None]
    
    z_data_1 = z_1_mesh[:, [i - 100 for i in data_depth_1]].flatten(order = 'F')[:, None]
    z_data_2 = z_2_mesh[:, data_depth_2].flatten(order = 'F')[:, None]
    
    t_int = Z_int[:number_int, 0:1]
    z_int = Z_int[:number_int, 1:2]


    # initialization
    tf.reset_default_graph()
    tf.set_random_seed(random_seed)
    random.seed(random_seed)
    np.random.seed(random_seed)

    # training    
    model = PINNs_RRE1D(
                t_res_1, z_res_1,
                t_res_2, z_res_2,
                t_int, z_int,
                t_data_1, z_data_1, theta_data_1,
                t_data_2, z_data_2, theta_data_2, 
                layers_psi_1, layers_psi_2, hydraulic_model, hydraulic_parameters_1, hydraulic_parameters_2,
                weight_values, normalize, ALR, LAA)

    # Adam training
    start_time = timeit.default_timer() 

    if RAR:
        iterations = iterations//10
        model.RAR(dom_coords, iterations, batch_size, new_residual_points)
    else:
        model.train_Adam(iterations, batch_size)
    time_Adam = timeit.default_timer() - start_time

    # BFGS training
    start_time = timeit.default_timer() 
    model.train_BFGS()
    time_BFGS = timeit.default_timer() - start_time

    # evaluation
    T = 201
    N = 101
    t_1 = np.linspace(dom_coords_1[0, 0], dom_coords_1[1, 0], T)[:, None]
    z_1 = np.linspace(dom_coords_1[0, 1], dom_coords_1[1, 1], N)[:, None]
    z_1_mesh, t_1_mesh = np.meshgrid(z_1, t_1)
    X_star_1 = np.hstack((t_1_mesh.flatten()[:, None], z_1_mesh.flatten()[:, None]))

    t_2 = np.linspace(dom_coords_2[0, 0], dom_coords_2[1, 0], T)[:, None]
    z_2 = np.linspace(dom_coords_2[0, 1], dom_coords_2[1, 1], N)[:, None]
    z_2_mesh, t_2_mesh = np.meshgrid(z_2, t_2)
    X_star_2 = np.hstack((t_2_mesh.flatten()[:, None], z_2_mesh.flatten()[:, None]))

    t_test_1 = X_star_1[:, 0:1]
    z_test_1 = X_star_1[:, 1:2]

    t_test_2 = X_star_2[:, 0:1]
    z_test_2 = X_star_2[:, 1:2]

    # prediction
    psi_1_pred, theta_1_pred, K_1_pred, flux_1_pred, residual_1 = model.predict_1(t_test_1, z_test_1)
    psi_2_pred, theta_2_pred, K_2_pred, flux_2_pred, residual_2 = model.predict_2(t_test_2, z_test_2)

    theta_1_pred = theta_1_pred.reshape((T, N))
    K_1_pred = K_1_pred.reshape((T, N))
    psi_1_pred = psi_1_pred.reshape((T, N))
    flux_1_pred = flux_1_pred.reshape((T, N))
    residual_1 = residual_1.reshape((T, N))

    theta_2_pred = theta_2_pred.reshape((T, N))
    K_2_pred = K_2_pred.reshape((T, N))
    psi_2_pred = psi_2_pred.reshape((T, N))
    flux_2_pred = flux_2_pred.reshape((T, N))
    residual_2 = residual_2.reshape((T, N))

    psi_pred = np.concatenate((psi_2_pred, psi_1_pred), axis = 1) 
    theta_pred = np.concatenate((theta_2_pred, theta_1_pred), axis = 1)
    K_pred = np.concatenate((K_2_pred, K_1_pred), axis = 1)
    residual = np.concatenate((residual_2, residual_1), axis = 1)
    flux_pred = np.concatenate((flux_2_pred, flux_1_pred), axis = 1) 

    t_mesh = np.concatenate((t_2_mesh, t_1_mesh), axis = 1)
    z_mesh = np.concatenate((z_2_mesh, z_1_mesh), axis = 1) 
        

    # import true solution: first index is depth, second index is time
    psi_true = np.load(f"../analytical_solutions/hydrus_example_psi.npy")
    theta_true = np.load(f"../analytical_solutions/hydrus_example_theta.npy")

    # the error at the interface is computed "twice" (theta in layer 1 and layer 2, which are not necesarily the same)
    theta_L2_error = np.sqrt(np.mean((theta_true - theta_pred) ** 2))
    theta_abs_error = np.mean(np.abs(theta_true- theta_pred))
    theta_square = np.sqrt(np.mean((theta_true) ** 2))
    theta_max_error = np.max(np.abs(theta_true- theta_pred))

    theta_L2_relative_error = theta_L2_error/theta_square
    theta_abs_relative_error = theta_abs_error/np.mean(np.abs(theta_true.T))

    print("theta_L2_relative_error is", theta_L2_relative_error)
    print("theta_abs_relative_error is", theta_abs_relative_error)
    print("theta_max_error is", theta_max_error)

    psi_L2_error = np.sqrt(np.mean((psi_true - psi_pred) ** 2))
    psi_abs_error = np.mean(np.abs(psi_true- psi_pred))
    psi_square = np.sqrt(np.mean((psi_true) ** 2))
    psi_max_error = np.max(np.abs(psi_true- psi_pred))

    psi_L2_relative_error = psi_L2_error/psi_square
    psi_abs_relative_error = psi_abs_error/np.mean(np.abs(psi_true.T))


    print("psi_L2_relative_error is", psi_L2_relative_error)
    print("psi_abs_relative_error is", psi_abs_relative_error)
    print("psi_max_error is", psi_max_error)
    
    # theta distribution
    fig, ax = plt.subplots(1, 3, figsize=(18, 5))

    c1 = ax[0].pcolor(t_mesh, z_mesh - 10.0, theta_pred, cmap='gnuplot2_r')
    fig.colorbar(c1, ax = ax[0])
    ax[0].set_xlabel(r'$t$ [hours]')
    ax[0].set_ylabel(r'$z$ [cm]')
    ax[0].set_title('Predicted $\\theta$')

    c2 = ax[1].pcolor(t_mesh, z_mesh - 10.0, theta_true, cmap='gnuplot2_r')
    fig.colorbar(c2, ax = ax[1])
    ax[1].set_xlabel(r'$t$ [hours]')
    ax[1].set_title('True $\\theta$')

    c3 = ax[2].pcolor(t_mesh, z_mesh - 10.0, np.abs(theta_pred - theta_true), cmap='gnuplot2_r')
    fig.colorbar(c3, ax = ax[2])
    ax[2].set_xlabel(r'$t$ [hours]')
    ax[2].set_title('Absolute error in $\\theta$')

    fig.savefig(f'../results/{folder}/{ID}/theta_color.png')
    fig.clf()

    # psi distribution
    fig, ax = plt.subplots(1, 3, figsize=(18, 5))

    c1 = ax[0].pcolor(t_mesh, z_mesh - 10.0, psi_pred, cmap='gnuplot2_r')
    fig.colorbar(c1, ax = ax[0])
    ax[0].set_xlabel(r'$t$ [hours]')
    ax[0].set_ylabel(r'$z$ [cm]')
    ax[0].set_title('Predicted $\\psi$ [cm]')

    c2 = ax[1].pcolor(t_mesh, z_mesh - 10.0, psi_true, cmap='gnuplot2_r')
    fig.colorbar(c2, ax = ax[1])
    ax[1].set_xlabel(r'$t$ [hours]')
    ax[1].set_title('True $\\psi$ [cm]')

    c3 = ax[2].pcolor(t_mesh, z_mesh - 10.0, np.abs(psi_pred - psi_true), cmap='gnuplot2_r')
    fig.colorbar(c3, ax = ax[2])
    ax[2].set_xlabel(r'$t$ [hours]')
    ax[2].set_title('Absolute error in $\\psi$ [cm]')

    fig.savefig(f'../results/{folder}/{ID}/psi_color.png')
    fig.clf()

    fig, ax = plt.subplots(1, 1, figsize=(6.5, 6.5))

    c1 = ax.pcolor(t_mesh, z_mesh - 10.0, flux_pred, cmap='gnuplot2_r')
    fig.colorbar(c1, ax = ax)
    ax.set_xlabel(r'$t$ [hours]')
    ax.set_ylabel(r'$z$ [cm]')
    ax.set_title('Predicted $q$ [cm/hours]')

    fig.savefig(f'../results/{folder}/{ID}/flux.png')
    fig.clf()

    fig = plt.subplots(1, 1, figsize=(6.5, 6.5))

    plt.scatter(theta_pred[0, ::5], z_mesh[0, ::5] - 10.0, s = 15, label = 'Predicted')
    plt.scatter(theta_pred[1, ::5], z_mesh[0, ::5] - 10.0, s = 15)
    plt.scatter(theta_pred[5, ::5], z_mesh[0, ::5] - 10.0, s = 15)
    plt.scatter(theta_pred[10, ::5], z_mesh[0, ::5] - 10.0, s = 15)
    plt.scatter(theta_pred[30, ::5], z_mesh[0, ::5] - 10.0, s = 15)
    plt.scatter(theta_pred[50, ::5], z_mesh[0, ::5] - 10.0, s = 15)
    plt.scatter(theta_pred[100, ::5], z_mesh[0, ::5] - 10.0, s = 15)
    plt.scatter(theta_pred[150, ::5], z_mesh[0, ::5] - 10.0, s = 15)
    plt.scatter(theta_pred[200, ::5], z_mesh[0, ::5] - 10.0, s = 15)

    plt.plot(theta_true[0, :], z_mesh[0, :] - 10.0 ,label = 't = 0 hour')
    plt.plot(theta_true[1, :], z_mesh[0, :] - 10.0,label = 't = 0.1 hour')
    plt.plot(theta_true[5, :], z_mesh[0, :] - 10.0,label = 't = 0.5 hour')
    plt.plot(theta_true[10, :], z_mesh[0, :]- 10.0 ,label = 't = 1 hour')
    plt.plot(theta_true[30, :], z_mesh[0, :]- 10.0,label = 't = 3 hour')
    plt.plot(theta_true[50, :], z_mesh[0, :]- 10.0,label = 't = 5 hour')
    plt.plot(theta_true[100, :], z_mesh[0, :]- 10.0, label = 't = 10 hour')
    plt.plot(theta_true[150, :], z_mesh[0, :]- 10.0, label = 't = 15 hour')
    plt.plot(theta_true[200, :], z_mesh[0, :]- 10.0, label = 't = 20 hour')

    plt.xlim(0.05,0.50)
    plt.xlabel('$\\theta$')
    plt.ylabel('$z$ cm')
    plt.legend(loc = (0.75,0.55), fontsize = 8)

    plt.savefig(f'../results/{folder}/{ID}/theta_fixed_time.png')
    plt.clf()

    t_res = np.concatenate((t_res_2, t_res_1), axis = 1)
    z_res = np.concatenate((z_res_2, z_res_1), axis = 1) 

    fig = plt.figure(1, figsize=(6.5, 6.5))

    plt.plot(t_res, z_res - 10.0, 'rx', markersize = 3, label = "residual points")

    if RAR:
        new_residual = model.new_residual[0]
        for i in range(1, len(model.new_residual)):
            new_residual = np.vstack((new_residual, model.new_residual[i]))
        plt.plot(new_residual[:, 0], new_residual[:, 1] - 10.0, 'bx', markersize = 5, label = "added residual points")



    plt.xlabel(r'$t$ [hours]')
    plt.ylabel(r'$z$ [cm]')
    plt.title('residual points')
    plt.xlim(-0.1, 10.1)
    plt.ylim(-10.1, 0.1)
    plt.legend(loc = (0.55,0.15))
    plt.savefig(f'../results/{folder}/{ID}/residual_points.png', transparent = True)
    plt.clf()

    fig = plt.figure(1, figsize=(6.5, 6.5))

    plt.pcolor(t_mesh, z_mesh - 10.0, np.log10(np.abs(residual)), cmap='gnuplot2_r')

    plt.colorbar()
    plt.xlabel(r'$t$ [hours]')
    plt.ylabel(r'$z$ [cm]')
    plt.title('$log_{10}r(x)$')
    plt.savefig(f'../results/{folder}/{ID}/residual.png', transparent = True)
    plt.clf()

    # Loss

    loss_res_1 = model.loss_res_1_log
    loss_data_1 = model.loss_data_1_log
    loss_res_2 = model.loss_res_2_log
    loss_data_2 = model.loss_data_2_log
    loss_psi_int = model.loss_psi_int_log
    loss_qz_int = model.loss_qz_int_log
    loss_res_int = model.loss_res_int_log

    Adam_iterations = np.arange(0, len(loss_res_1)) * 10

    loss_res_1_BFGS = model.loss_res_1_log_BFGS
    loss_data_1_BFGS = model.loss_data_1_log_BFGS
    loss_res_2_BFGS = model.loss_res_2_log_BFGS
    loss_data_2_BFGS = model.loss_data_2_log_BFGS
    loss_psi_int_BFGS = model.loss_psi_int_log_BFGS
    loss_qz_int_BFGS = model.loss_qz_int_log_BFGS
    loss_res_int_BFGS = model.loss_res_int_log_BFGS

    BFGS_iterations = np.arange(0, len(loss_res_1_BFGS))

    total_iterations = np.append(Adam_iterations, BFGS_iterations + Adam_iterations[-1])

    fig, ax = plt.subplots(1, 3, figsize=(18, 5))

    # Adam
    ax[0].plot(Adam_iterations, loss_data_1, label='$\mathcal{L}_{data1}$')
    ax[0].plot(Adam_iterations, loss_data_2, label='$\mathcal{L}_{data2}$')
    ax[0].plot(Adam_iterations, loss_res_1, label='$\mathcal{L}_{res1}$')
    ax[0].plot(Adam_iterations, loss_res_2, label='$\mathcal{L}_{res2}$')
    ax[0].plot(Adam_iterations, loss_psi_int, label='$\mathcal{L}_{\psi-int}$')
    ax[0].plot(Adam_iterations, loss_qz_int, label='$\mathcal{L}_{qz-int}$')
    ax[0].plot(Adam_iterations, loss_res_int, label='$\mathcal{L}_{res-int}$')
    ax[0].set_yscale('log')
    ax[0].set_xlabel('iterations')
    ax[0].set_ylabel('Loss')
    ax[0].set_title('Adam')
    ax[0].legend()

    # BFGS
    ax[1].plot(BFGS_iterations, loss_data_1_BFGS, label='$\mathcal{L}_{data1}$')
    ax[1].plot(BFGS_iterations, loss_data_2_BFGS, label='$\mathcal{L}_{data2}$')
    ax[1].plot(BFGS_iterations, loss_res_1_BFGS, label='$\mathcal{L}_{res1}$')
    ax[1].plot(BFGS_iterations, loss_res_2_BFGS, label='$\mathcal{L}_{res2}$')
    ax[1].plot(BFGS_iterations, loss_psi_int_BFGS, label='$\mathcal{L}_{\psi-int}$')
    ax[1].plot(BFGS_iterations, loss_qz_int_BFGS, label='$\mathcal{L}_{qz-int}$')
    ax[1].plot(BFGS_iterations, loss_res_int_BFGS, label='$\mathcal{L}_{res-int}$')

    ax[1].set_yscale('log')
    ax[1].set_xlabel('iterations')
    ax[1].set_ylabel('Loss')
    ax[1].set_title('L-BFGS-B')
    ax[1].legend()


    # total training
    ax[2].plot(total_iterations, loss_data_1 + loss_data_1_BFGS, label='$\mathcal{L}_{data1}$')
    ax[2].plot(total_iterations, loss_data_2 + loss_data_2_BFGS, label='$\mathcal{L}_{data2}$')
    ax[2].plot(total_iterations, loss_res_1 + loss_res_1_BFGS, label='$\mathcal{L}_{res1}$')
    ax[2].plot(total_iterations, loss_res_2 + loss_res_2_BFGS, label='$\mathcal{L}_{res2}$')
    ax[2].plot(total_iterations, loss_psi_int + loss_psi_int_BFGS, label='$\mathcal{L}_{\psi-int}$')
    ax[2].plot(total_iterations, loss_qz_int + loss_qz_int_BFGS, label='$\mathcal{L}_{qz-int}$')
    ax[2].plot(total_iterations, loss_res_int + loss_res_int_BFGS, label='$\mathcal{L}_{res-int}$')


    ax[2].set_yscale('log')
    ax[2].set_xlabel('iterations')
    ax[2].set_ylabel('Loss')
    ax[2].set_title('total')
    ax[2].legend()

    plt.savefig(f'../results/{folder}/{ID}/training.png')
    plt.clf()

    np.save(f"../results/{folder}/{ID}/theta_pred", theta_pred)
    np.save(f"../results/{folder}/{ID}/psi_pred", psi_pred)
    np.save(f"../results/{folder}/{ID}/K_pred", K_pred)
    np.save(f"../results/{folder}/{ID}/flux_pred", flux_pred)
    np.save(f"../results/{folder}/{ID}/residual", residual)

    np.save(f"../results/{folder}/{ID}/loss_data_1", loss_data_1)
    np.save(f"../results/{folder}/{ID}/loss_data_1_BFGS", loss_data_1_BFGS)
    np.save(f"../results/{folder}/{ID}/loss_data_2", loss_data_1)
    np.save(f"../results/{folder}/{ID}/loss_data_2_BFGS", loss_data_2_BFGS)

    np.save(f"../results/{folder}/{ID}/loss_res_1", loss_res_1)
    np.save(f"../results/{folder}/{ID}/loss_res_1_BFGS", loss_res_1_BFGS)
    np.save(f"../results/{folder}/{ID}/loss_res_2", loss_res_2)
    np.save(f"../results/{folder}/{ID}/loss_res_2_BFGS", loss_res_2_BFGS)

    np.save(f"../results/{folder}/{ID}/loss_psi_int", loss_psi_int)
    np.save(f"../results/{folder}/{ID}/loss_psi_int_BFGS", loss_psi_int_BFGS)
    np.save(f"../results/{folder}/{ID}/loss_qz_int", loss_qz_int)
    np.save(f"../results/{folder}/{ID}/loss_qz_int_BFGS", loss_qz_int_BFGS)
    np.save(f"../results/{folder}/{ID}/loss_res_int", loss_res_int)
    np.save(f"../results/{folder}/{ID}/loss_res_int_BFGS", loss_res_int_BFGS)


    np.save(f"../results/{folder}/{ID}/t_res", t_res)
    np.save(f"../results/{folder}/{ID}/z_res", z_res)
    if RAR:
        np.save(f"../results/{folder}/{ID}/new_residual", new_residual)

    np.save(f"../results/{folder}/{ID}/z_mesh", z_mesh)
    np.save(f"../results/{folder}/{ID}/t_mesh", t_mesh)


    sheet1.write(ID, 0, ID)
    sheet1.write(ID, 1, random_seed)
    sheet1.write(ID, 2, number_layers_1)
    sheet1.write(ID, 3, number_units_1)
    sheet1.write(ID, 4, number_layers_2)
    sheet1.write(ID, 5, number_units_2)
    sheet1.write(ID, 6, LAA)
    sheet1.write(ID, 7, number_res_1)
    sheet1.write(ID, 8, number_res_2)
    sheet1.write(ID, 9, str(data_depth))
    sheet1.write(ID, 10, number_int)

    sheet1.write(ID, 11, weight_values[0])
    sheet1.write(ID, 12, weight_values[1])
    sheet1.write(ID, 13, weight_values[2])
    sheet1.write(ID, 14, weight_values[3])
    sheet1.write(ID, 15, weight_values[4])
    sheet1.write(ID, 16, weight_values[5])

    sheet1.write(ID, 17, ALR)
    sheet1.write(ID, 18, iterations)
    sheet1.write(ID, 19, batch_size)
    sheet1.write(ID, 20, RAR)
    sheet1.write(ID, 21, new_residual_points)
    sheet1.write(ID, 22, time_Adam)
    sheet1.write(ID, 23, time_BFGS)

    sheet1.write(ID, 24, theta_L2_relative_error)
    sheet1.write(ID, 25, theta_abs_relative_error)
    sheet1.write(ID, 26, theta_max_error)
    sheet1.write(ID, 27, psi_L2_relative_error)
    sheet1.write(ID, 28, psi_abs_relative_error)
    sheet1.write(ID, 29, psi_max_error)

    sheet1.write(ID, 30, str(loss_data_1_BFGS[-1]))
    sheet1.write(ID, 31, str(loss_data_2_BFGS[-1]))
    sheet1.write(ID, 32, str(loss_res_1_BFGS[-1]))
    sheet1.write(ID, 33, str(loss_res_2_BFGS[-1]))
    sheet1.write(ID, 34, str(loss_res_int_BFGS[-1]))
    sheet1.write(ID, 35, str(loss_psi_int_BFGS[-1]))
    sheet1.write(ID, 36, str(loss_qz_int_BFGS[-1]))


In [ ]:
folder = "example-inverse"
wb = xlwt.Workbook()
sheet1 = wb.add_sheet(f'{folder}')
sheet1.write(0, 0,"ID")
sheet1.write(0, 1, "random_seed")
sheet1.write(0, 2, "number_layers_1")
sheet1.write(0, 3, "number_units_1")
sheet1.write(0, 4, "number_layers_2")
sheet1.write(0, 5, "number_units_2")
sheet1.write(0, 6, "LAA")
sheet1.write(0, 7, "number_res_1")
sheet1.write(0, 8, "number_res_2")
sheet1.write(0, 9, "data_depth")
sheet1.write(0, 10, "number_int")
sheet1.write(0, 11, "weight_values_data_1")
sheet1.write(0, 12, "weight_values_data_2")
sheet1.write(0, 13, "weight_values_res_2")
sheet1.write(0, 14, "weight_values_int")
sheet1.write(0, 15, "weight_values_psi_int")
sheet1.write(0, 16, "weight_values_qz_int")
sheet1.write(0, 17, "ALR")
sheet1.write(0, 18, "iterations")
sheet1.write(0, 19, "batch_size")
sheet1.write(0, 20, "RAR")
sheet1.write(0, 21, "new_residual_points")
sheet1.write(0, 22, "time_Adam")
sheet1.write(0, 23, "time_BFGS")
sheet1.write(0, 24, "theta_L2_relative_error")
sheet1.write(0, 25, "theta_abs_relative_error")
sheet1.write(0, 26, "theta_max_error")
sheet1.write(0, 27, "psi_L2_relative_error")
sheet1.write(0, 28, "psi_abs_relative_error")
sheet1.write(0, 29, "psi_max_error")
sheet1.write(0, 30,  "loss_data_1")
sheet1.write(0, 31,  "loss_data_2")
sheet1.write(0, 32,  "loss_res_1")
sheet1.write(0, 33,  "loss_res_2")
sheet1.write(0, 34,  "loss_res_int")
sheet1.write(0, 35,  "loss_psi_int")
sheet1.write(0, 36,  "loss_qz_int")

random_seed = 1
data_depth_list = [[10, 50, 90, 130, 170]] # measurement data at -1, -5, -9, -13, -17 cm
ID = 1

for i in range(len(random_seed_list)):
    for j in range(len(data_depth_list)):
        main_loop(folder, ID, random_seed, data_depth_list[j])
        ID += 1
        wb.save(f'../results/inverse/{folder}/summary.xls')